# 特征

In [1]:
import pandas as pd

df_feature = pd.read_parquet("data/features_d002.parquet")
df_label = pd.read_parquet("data/label_d002.parquet")
df_label.head()

,close,ret,trgt,bin,side
index,,,,,
2021-01-01 00:00:00+00:00,29016.23,NaN,NaN,0.0,0.0
2021-01-01 00:03:00+00:00,28965.02,NaN,NaN,0.0,0.0
2021-01-01 00:06:00+00:00,28928.06,NaN,NaN,0.0,0.0
2021-01-01 00:09:00+00:00,28851.82,NaN,NaN,0.0,0.0
2021-01-01 00:12:00+00:00,28786.75,NaN,NaN,0.0,0.0


# 模型

In [2]:
import json

with open("data/mrmr_meta_features.json", "r") as f:
    mrmr_meta_features = json.load(f)

with open("data/mrmr_side_features.json", "r") as f:
    mrmr_side_features = json.load(f)

meta_features_col = [k for k, v in mrmr_meta_features.items() if v > 0]
len(meta_features_col)

1263

In [3]:
invalid_len = 300

df_feature = df_feature.iloc[invalid_len:]
df_label = df_label.iloc[invalid_len:]

train_split_point = int(len(df_feature) * 0.8)
meta_train_feature = df_feature.iloc[:train_split_point][meta_features_col]
meta_train_label = df_label.iloc[:train_split_point]
meta_train_feature = meta_train_feature[meta_train_label["ret"].notna().tolist()]
meta_train_label = meta_train_label[meta_train_label["ret"].notna().tolist()][
    "bin"
].astype(int)
assert meta_train_feature.shape[0] == meta_train_label.shape[0]
print(meta_train_feature.shape)

meta_test_feature = df_feature.iloc[train_split_point:][meta_features_col]
meta_test_label = df_label.iloc[train_split_point:]
meta_test_feature = meta_test_feature[meta_test_label["ret"].notna().tolist()]
meta_test_label = meta_test_label[meta_test_label["ret"].notna().tolist()][
    "bin"
].astype(int)
assert meta_test_feature.shape[0] == meta_test_label.shape[0]
print(meta_test_feature.shape)

side_train_feature = df_feature.iloc[:train_split_point]
side_train_label = df_label.iloc[:train_split_point]
side_train_feature = side_train_feature[
    (side_train_label["side"].astype(int) != 0).tolist()
]
side_train_label = side_train_label[
    (side_train_label["side"].astype(int) != 0).tolist()
]["side"].astype(int)
side_train_label = (side_train_label == 1).astype(int)
assert side_train_feature.shape[0] == side_train_label.shape[0]
print(side_train_feature.shape)

side_test_feature = df_feature.iloc[train_split_point:]
side_test_label = df_label.iloc[train_split_point:]
side_test_feature = side_test_feature[
    (side_test_label["side"].astype(int) != 0).tolist()
]
side_test_label = side_test_label[(side_test_label["side"].astype(int) != 0).tolist()][
    "side"
].astype(int)
side_test_label = (side_test_label == 1).astype(int)
assert side_test_feature.shape[0] == side_test_label.shape[0]
print(side_test_feature.shape)

(427148, 1263)
(106939, 1263)
(220169, 1263)
(57137, 1263)


In [4]:
meta_train_label.value_counts()

bin
1    220169
0    206979
Name: count, dtype: int64

In [5]:
import optuna_integration.lightgbm as lgb
from lightgbm import log_evaluation

data_meta_train = lgb.Dataset(meta_train_feature, meta_train_label)
data_meta_test = lgb.Dataset(meta_test_feature, meta_test_label)

params = {
    "objective": "binary",
    "is_unbalance": True,
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "num_threads": -1,
}
model = lgb.train(
    params,
    data_meta_train,
    num_boost_round=1000,
    valid_sets=data_meta_test,
    callbacks=[log_evaluation(1000)],
)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-02-16 13:39:51,326] A new study created in memory with name: no-name-a4d89105-a0a0-4515-8627-19253a03f82c
feature_fraction, val_score: 0.724939:  14%|#4        | 1/7 [02:16<13:41, 136.85s/it]

[1000]	valid_0's auc: 0.724939


feature_fraction, val_score: 0.726656:  29%|##8       | 2/7 [03:45<09:00, 108.15s/it]

[1000]	valid_0's auc: 0.726656


feature_fraction, val_score: 0.726656:  43%|####2     | 3/7 [05:00<06:13, 93.44s/it]

...

[1000]	valid_0's auc: 0.72489


num_leaves, val_score: 0.727815:  75%|#######5  | 15/20 [35:30<09:54, 118.89s/it]

[1000]	valid_0's auc: 0.725682


[W 2025-02-16 14:29:35,413] Trial 22 failed with parameters: {'num_leaves': 83} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/optuna_integration/_lightgbm_tuner/optimize.py", line 238, in __call__
    booster = lgb.train(self.lgbm_params, train_set, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/lightgbm/engine.py", line 307, in train
    booster.update(fobj=fobj)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/lightgbm/basic.py", line 4136, in update
    _LIB.LGBM_BoosterUpdateOneIter(
KeyboardInterrupt
[W 202

KeyboardInterrupt: 

In [ ]:
model.params